In [4]:
import pandas as pd
import json
import plotly.express as px

In [2]:
properties_file = open('../hazard_mitigation_properties.json', 'r')
projects_file = open('../hazard_mitigation_projects.json', 'r')
properties_json = json.load(properties_file)
projects_json = json.load(projects_file)

df_properties = pd.DataFrame(properties_json['HazardMitigationAssistanceMitigatedProperties'])
df_projects = pd.DataFrame(projects_json['HazardMitigationAssistanceProjects'])

In [3]:
df_properties.head()

,projectIdentifier,programArea,programFy,disasterNumber,propertyAction,structureType,typeOfResidency,foundationType,county,city,state,stateNumberCode,region,zip,damageCategory,actualAmountPaid,numberOfProperties,id
0,DR-4022-0031-R,HMGP,2011,4022.0,Acquisition/Demolition,Non-residential - Private,Other (Specify in Comments),Slab on Grade,Windham,WILMINGTON,Vermont,50,1.0,05363,50 - 99%,NaN,1,23533667-6a8a-4779-b28f-5c4d825ef426
1,DR-1976-0020-R,HMGP,2011,1976.0,Acquisition/Demolition,Single Family,Rental,Basement,Jefferson,LOUISVILLE,Kentucky,21,4.0,40211,N/A,63999.0,1,197d75e1-34c8-436a-ba1e-3adc895ca1f8
2,DR-1971-0209-R,HMGP,2011,1971.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Crawl Space,Jefferson,CONCORD,Alabama,1,4.0,35203,N/A,4000.0,1,44546cd7-20f7-4dd4-ad00-7baf4b8d750f
3,DR-1971-0153-R,HMGP,2011,1971.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Slab on Grade,Marshall,GUNTERSVILLE,Alabama,1,4.0,35976,N/A,4900.0,1,f3a0ceed-f9e4-4ccb-9a5f-e13595355823
4,DR-1917-0013-R,HMGP,2010,1917.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Slab on Grade,Canadian,YUKON,Oklahoma,40,6.0,73099,N/A,2000.0,20,de1dc299-3c03-48c9-b36f-2eb45645d000


In [21]:
df_projects.head()

,projectIdentifier,programArea,programFy,region,state,stateNumberCode,county,countyCode,disasterNumber,projectCounties,...,subrecipientAdminCostAmt,srmcObligatedAmt,recipientAdminCostAmt,costSharePercentage,benefitCostRatio,netValueBenefits,numberOfFinalProperties,numberOfProperties,id,disaster based
0,DR-1961-0002-R,HMGP,2011,7,Missouri,29,Moniteau,135,1961.0,MONITEAU,...,0.0,0.0,0.0,0.749996,2.175,500586.0,1,1,d9c99d55-5a60-40ea-a758-3a15de021115,True
1,DR-1751-0018-P,HMGP,2008,6,Arkansas,5,Statewide,0,1751.0,STATEWIDE,...,0.0,0.0,0.0,0.750000,0.000,0.0,0,0,1627e572-9f79-4055-9ede-40cf3226c5f6,True
2,DR-1840-0002-R,HMGP,2009,4,Florida,12,Volusia,127,1840.0,VOLUSIA,...,0.0,0.0,0.0,0.750000,1.307,516759.0,0,0,08ea52a7-5126-45a1-aedb-5a5222fbc17c,True
3,DR-1545-0065-R,HMGP,2004,4,Florida,12,Volusia,127,1545.0,VOLUSIA,...,0.0,0.0,0.0,0.750000,69.041,930678.0,0,0,512a4ab1-7ca1-45a3-8390-92b9ef26a07f,True
4,DR-4030-0016-R,HMGP,2011,3,Pennsylvania,42,Columbia,37,4030.0,COLUMBIA,...,0.0,0.0,0.0,0.750000,0.000,0.0,5,6,84e82525-63c5-48b9-8cb6-9e8528171081,True


In [22]:
# check if the property and project are because of a disaster or not, more info on 
# https://www.fema.gov/openfema-data-page/hazard-mitigation-assistance-mitigated-properties-v3
# https://www.fema.gov/openfema-data-page/hazard-mitigation-assistance-projects-v3
df_properties['disaster based'] = df_properties.apply(lambda row: 'DR' in row['projectIdentifier'], axis=1)
df_projects['disaster based'] = df_projects.apply(lambda row: 'DR' in row['projectIdentifier'], axis=1)

In [5]:
df_properties.to_json('../preprocessed-fema-properties.json')
df_projects.to_json('../preprocessed-fema-projects.json')

In [25]:
states = df_properties.groupby(['state', 'programFy', 'programArea'], as_index=False).sum(numeric_only=True)
states.head()

,state,programFy,programArea,disasterNumber,region,actualAmountPaid,numberOfProperties,disaster based
0,Alabama,1990,HMGP,2583.0,12.0,0.0,11,3
1,Alabama,1994,HMGP,16442.0,64.0,0.0,97,16
2,Alabama,1995,HMGP,4188.0,16.0,0.0,5,4
3,Alabama,1996,HMGP,55016.0,204.0,0.0,343,51
4,Alabama,1997,FMA,0.0,12.0,0.0,3,0


In [2]:
df_disasters = pd.read_csv('../Preprocessed-Natural-Disasters.csv', delimiter=';')
df_disasters_us = df_disasters[df_disasters['ISO'] == 'USA']

In [3]:
df_properties = pd.read_json('../preprocessed-fema-properties.json')

In [4]:
df_properties[df_properties['damageCategory'] != 'N/A']['damageCategory']

0        50 - 99%
15       50 - 99%
17        0 - 49%
19        0 - 49%
35       50 - 99%
           ...   
69731     0 - 49%
69732     0 - 49%
69745        100%
69746    50 - 99%
69749    50 - 99%
Name: damageCategory, Length: 26846, dtype: object

In [5]:
def get_fema_cost_distribution(state,year,category):
    if (state):
        data = df_properties[df_properties['state'] == state]
    else:
        data = df_properties

    categories = data.groupby(category, as_index=False).sum(numeric_only=True)
    total_spent = sum(categories['actualAmountPaid'].values)      
    categories['spent percentage'] = categories.apply(lambda x: x['actualAmountPaid']/total_spent, axis=1)
    print(categories[[category, 'spent percentage']])
    return categories

In [6]:
get_fema_cost_distribution(None,None,'propertyAction')['spent percentage']

                      propertyAction  spent percentage
0         Acquisition of Vacant Land      1.521470e-03
1             Acquisition/Demolition      5.658740e-01
2             Acquisition/Relocation      7.317760e-04
3                          Elevation      7.860889e-02
4                       Floodproofed      6.243115e-03
5          Mitigation Reconstruction      6.580360e-04
6        Other (Specify in Comments)      2.585890e-03
7            Retro (1/15/2008 - DSG)      1.488133e-05
8             Safe Room/Wind Shelter      2.471158e-01
9                   Seismic Retrofit      3.566646e-03
10                 Wildfire Retrofit      1.650698e-04
11                     Wind Retrofit      9.291418e-02
12    Wind Retrofit Advanced(A-P804)      1.608323e-08
13      Wind Retrofit Basic (B-P804)      2.636042e-07
14  Wind Retrofit Estimated(ES-P804)      0.000000e+00


0     1.521470e-03
1     5.658740e-01
2     7.317760e-04
3     7.860889e-02
4     6.243115e-03
5     6.580360e-04
6     2.585890e-03
7     1.488133e-05
8     2.471158e-01
9     3.566646e-03
10    1.650698e-04
11    9.291418e-02
12    1.608323e-08
13    2.636042e-07
14    0.000000e+00
Name: spent percentage, dtype: float64

In [12]:
def compare_with_other_country(category_a,category_b,country):
    grouped = df_properties.groupby('programFy', as_index=False).sum(numeric_only=True)
    grouped['actualAmountPaid'] = grouped['actualAmountPaid'].cumsum()
    disasters_grouped = df_disasters_us.groupby('Start Year', as_index=False).sum(numeric_only=True)
    disasters_grouped["Total Damages, Adjusted ('000 US$)"] = disasters_grouped["Total Damages, Adjusted ('000 US$)"].cumsum()

    def get_prevention_costs(row):
        prevention_row = grouped[grouped['programFy'] == row['Start Year']]
        if (prevention_row.empty):
            return 0.
        return prevention_row['actualAmountPaid']

    def get_total_costs(row):
        country_disasters = df_disasters[df_disasters['ISO'] != "USA"]
        country_grouped = country_disasters.groupby('Start Year', as_index=False).sum(numeric_only=True) #.sum(numeric_only=True)
        country_grouped["Total Damages, Adjusted ('000 US$)"] = country_grouped["Total Damages, Adjusted ('000 US$)"].cumsum()
        other_country_row = country_grouped[country_grouped['Start Year'] == row['Start Year']]
        if other_country_row.empty:
            return 0
        else:
            # print(other_country_row)
            return other_country_row["Total Damages, Adjusted ('000 US$)"].mean()


    disasters_grouped['prevention costs'] = disasters_grouped.apply(get_prevention_costs, axis=1)
    disasters_grouped['prevention costs'] = disasters_grouped['prevention costs'].astype(dtype=float)
    disasters_grouped['other country'] = disasters_grouped.apply(get_total_costs, axis=1)
    disasters_grouped['other country'] = disasters_grouped['other country'].astype(dtype=float)

    return px.line(disasters_grouped[disasters_grouped['Start Year'] < 2023], 'Start Year', ["Total Damages, Adjusted ('000 US$)", 'other country'])


fig = compare_with_other_country(None,None,'BEL')
fig.show()

C:\Users\rafae\AppData\Local\Temp\ipykernel_45952\2289100761.py:27: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



0     8.876529e+06
1     9.585362e+06
2     2.099164e+07
3     3.899616e+07
4     4.871793e+07
          ...     
59    4.016840e+09
60    4.126553e+09
61    4.250997e+09
62    4.328399e+09
63    4.328399e+09
Name: Total Damages, Adjusted ('000 US$), Length: 64, dtype: float64
0     8.876529e+06
1     9.585362e+06
2     2.099164e+07
3     3.899616e+07
4     4.871793e+07
          ...     
59    4.016840e+09
60    4.126553e+09
61    4.250997e+09
62    4.328399e+09
63    4.328399e+09
Name: Total Damages, Adjusted ('000 US$), Length: 64, dtype: float64
0     8.876529e+06
1     9.585362e+06
2     2.099164e+07
3     3.899616e+07
4     4.871793e+07
          ...     
59    4.016840e+09
60    4.126553e+09
61    4.250997e+09
62    4.328399e+09
63    4.328399e+09
Name: Total Damages, Adjusted ('000 US$), Length: 64, dtype: float64
0     8.876529e+06
1     9.585362e+06
2     2.099164e+07
3     3.899616e+07
4     4.871793e+07
          ...     
59    4.016840e+09
60    4.126553e+09
61    4.250997

In [21]:
def get_total_costs(row):
    country = 'BRA'
    country_disasters = df_disasters[df_disasters['ISO'] == country]
    country_grouped = country_disasters.groupby('Start Year', as_index=False).sum(numeric_only=True)
    country_grouped["Total Damages, Adjusted ('000 US$)"] = country_grouped["Total Damages, Adjusted ('000 US$)"].cumsum()
    return country_grouped

idk = get_total_costs(None)
idk.head()
    # other_country_row = country_grouped[country_grouped['Start Year'] == row['Start Year']]
    # print(country_grouped["Total Damages, Adjusted ('000 US$)"])
    # if other_country_row.empty:
    #     return 0
    # else:
    #     print(other_country_row)
    #     return other_country_row["Total Damages, Adjusted ('000 US$)"]

,Start Year,AID Contribution ('000 US$),Dis Mag Value,Latitude,Longitude,Start Month,Start Day,End Year,End Month,End Day,...,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)"
0,1964,0.0,0.0,-19.615441,-78.602024,12.0,5.0,5892,12.0,5.0,...,0.0,120000.0,0.0,120000.0,0.0,0.0,0.0,0.0,500.0,4718.0
1,1965,0.0,0.0,-32.676530,-86.557830,14.0,16.0,3930,14.0,16.0,...,0.0,230000.0,40000.0,270000.0,0.0,0.0,0.0,0.0,17000.0,162617.0
2,1966,0.0,0.0,-56.480459,-126.557049,11.0,43.0,9830,11.0,43.0,...,1757.0,4177381.0,29000.0,4208138.0,0.0,0.0,0.0,0.0,60941.0,712080.0
3,1967,0.0,0.0,-101.678013,-221.486585,28.0,120.0,9835,28.0,120.0,...,1307.0,172836.0,15400.0,189543.0,0.0,0.0,0.0,0.0,15300.0,846308.0
4,1968,0.0,0.0,0.000000,0.000000,12.0,0.0,1968,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,846308.0


In [20]:
def fill_missing_columns_with_default(df, columns, columns_to_fill, values):
    copy_df = df.copy()
    unique_columns = [copy_df[col].unique() for col in columns]
    index = pd.MultiIndex.from_product(unique_columns,names=columns)
    new_df = pd.DataFrame(index=index, columns=['count'])
    merged_df = pd.merge(copy_df, new_df, how='right', left_on=columns, right_index=True)
    for idx,col in enumerate(columns_to_fill):
        merged_df[col] = merged_df[col].fillna(values[idx])
    return merged_df

def get_gdp_data(df_gdp: pd.DataFrame, df_disaster: pd.DataFrame, years, country = None, categories = False):
    columns = years + ['Country Code']
    columns = [str(c) for c in columns]
    gdp_data_filtered = df_gdp[columns]
    if (country):
        gdp_data_reduced = gdp_data_filtered[gdp_data_filtered['Country Code'] == country]
        disaster_columns = ['Start Year', 'ISO']
        if (categories): disaster_columns += ['Disaster Subgroup', 'Disaster Type']
        disaster_data_by_year = df_disaster.groupby(disaster_columns, as_index=False).sum(numeric_only=True)
        disaster_data_by_year = disaster_data_by_year[disaster_data_by_year['ISO'] == country] 
    else:
        disaster_columns = ['Start Year']
        if (categories): disaster_columns += ['Disaster Subgroup', 'Disaster Type']
        gdp_data_reduced = pd.DataFrame([gdp_data_filtered.mean(numeric_only=True)], columns=[str(y) for y in years])
        disaster_data_by_year = df_disaster.groupby(disaster_columns, as_index=False).sum(numeric_only=True)
        disaster_data_by_year['ISO'] = 'WORLD'

    def calculate_gdp_share(row):
        year = str(int(row['Start Year']))
        if (not (year in gdp_data_reduced.columns)):
            return 0
        gdp = gdp_data_reduced[year]
        if (gdp.empty):
            return 0
        damages = row["Total Damages, Adjusted ('000 US$)"] * 1000
        return (damages / gdp) * 100

    columns_to_fill = ['Start Year', 'ISO', 'Disaster Subgroup', 'Disaster Type']

    if (disaster_data_by_year.empty):
        disaster_data_by_year["share"] = 0
    else:
        disaster_data_by_year['share'] = disaster_data_by_year.apply(calculate_gdp_share, axis=1)

    if categories:
        filled_df = fill_missing_columns_with_default(disaster_data_by_year,columns_to_fill,["Total Damages, Adjusted (\'000 US$)", 'share'],[0,0])
        return filled_df
    return disaster_data_by_year

In [29]:
def fill_years(df,years,country, disaster_classification): 
    all_combinations = [(year, country, disaster_subgroup, disaster_type) for disaster_subgroup, disaster_types in disaster_classification.items() for disaster_type in disaster_types for year in years]
    all_combinations_df = pd.DataFrame(all_combinations, columns=['Start Year', 'ISO', 'Disaster Subgroup', 'Disaster Type'])
    merged_df = pd.merge(df, all_combinations_df,on=['Start Year', 'ISO', 'Disaster Subgroup', 'Disaster Type'], how="right")
    return merged_df
    # merged_df.reset_index()

In [30]:
df_gdp = pd.read_csv('../gdp_data_constant.csv')
df_disaster = pd.read_csv('../Preprocessed-Natural-Disasters.csv',delimiter=';')

disaster_subgroups = df_disaster['Disaster Subgroup'].unique()
disaster_types = df_disaster['Disaster Type'].unique()

disaster_classification = {dis_subgroup: list(df_disaster[df_disaster["Disaster Subgroup"] == dis_subgroup]["Disaster Type"].unique()) for dis_subgroup in df_disaster['Disaster Subgroup'].unique()}

isos = df_disaster['ISO'].unique()

years = list(range(1960,2023))

total_gdp_df = pd.DataFrame()
for iso in isos:
    gdp_data = get_gdp_data(df_gdp,df_disaster,years,country=iso,categories=True)
    gdp_data = gdp_data[['Start Year', 'ISO', 'Disaster Subgroup', 'Disaster Type','share']]
    gdp_data = fill_years(gdp_data,years,iso, disaster_classification)
    gdp_data.fillna(0,inplace=True)
    total_gdp_df = pd.concat([total_gdp_df, gdp_data])

      Start Year  ISO Disaster Subgroup      Disaster Type   
5           1960  CHL       Geophysical         Earthquake  \
115         1964  CHL       Geophysical  Volcanic activity   
168         1965  CHL       Geophysical         Earthquake   
169         1965  CHL      Hydrological              Flood   
170         1965  CHL      Hydrological          Landslide   
...          ...  ...               ...                ...   
7554        2019  CHL       Geophysical         Earthquake   
7555        2019  CHL      Hydrological              Flood   
7965        2021  CHL      Hydrological              Flood   
7966        2021  CHL    Meteorological              Storm   
8171        2022  CHL    Climatological           Wildfire   

      AID Contribution ('000 US$)  Dis Mag Value   Latitude  Longitude   
5                             0.0           10.0 -41.476041 -72.810417  \
115                           0.0            0.0 -39.429426 -72.138818   
168                           0.0

In [31]:
total_gdp_df.to_csv('../gdp_data.csv', index=False)